<a href="https://colab.research.google.com/github/Sankytanky100/Data-Engineering/blob/main/Pyspark_in_Colab_Tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook Overview
This tutorial introduces PySpark fundamentals in Google Colab, focusing on environment setup, DataFrame creation, and core transformations/actions.

**You will learn**
- How to initialize a Spark session in Colab.
- Common DataFrame operations (select, filter, withColumn, aggregations).
- How to inspect and validate results at each step.

**Prerequisites**
- Colab runtime with internet access.
- PySpark installation (handled in the setup cells).


In [1]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
!pip install pyspark


In [2]:
# Check Java version
!java -version

# Check PySpark version
!pyspark --version


openjdk version "11.0.25" 2024-10-15
OpenJDK Runtime Environment (build 11.0.25+9-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.25+9-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.3
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 11.0.25
Branch HEAD
Compiled by user haejoon.lee on 2024-09-09T05:20:05Z
Revision 32232e9ed33bb16b93ad58cfde8b82e0f07c0970
Url https://github.com/apache/spark
Type --help for more information.


In [3]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("PySpark on Google Colab") \
    .getOrCreate()

# Verify SparkSession
print("SparkSession created:", spark)


SparkSession created: <pyspark.sql.session.SparkSession object at 0x7912c237a470>


In [4]:
student_data = [("Chris",1523,0.72,"CA"),
                ("Jake", 1555,0.83,"NY"),
                ("Cody", 1439,0.92,"CA"),
                ("Lisa",1442,0.81,"FL"),
                ("Daniel",1600,0.88,"TX"),
                ("Kelvin",1382,0.99,"FL"),
                ("Nancy",1442,0.74,"TX"),
                ("Pavel",1599,0.82,"NY"),
                ("Josh",1482,0.78,"CA"),
                ("Cynthia",1582,0.94,"CA")]
spark = SparkSession.builder.getOrCreate()
student_rdd = spark.sparkContext.parallelize(student_data, 5)

In [5]:
# confirm your RDD contains the correct data
student_rdd.collect()

[('Chris', 1523, 0.72, 'CA'),
 ('Jake', 1555, 0.83, 'NY'),
 ('Cody', 1439, 0.92, 'CA'),
 ('Lisa', 1442, 0.81, 'FL'),
 ('Daniel', 1600, 0.88, 'TX'),
 ('Kelvin', 1382, 0.99, 'FL'),
 ('Nancy', 1442, 0.74, 'TX'),
 ('Pavel', 1599, 0.82, 'NY'),
 ('Josh', 1482, 0.78, 'CA'),
 ('Cynthia', 1582, 0.94, 'CA')]

In [6]:
student_rdd.getNumPartitions()

5

In [7]:
rdd_transformation = student_rdd.map(lambda x: (x[0], x[1], int(x[2]*100), x[3]))

# confirm transformation is correct
rdd_transformation.collect()

[('Chris', 1523, 72, 'CA'),
 ('Jake', 1555, 83, 'NY'),
 ('Cody', 1439, 92, 'CA'),
 ('Lisa', 1442, 81, 'FL'),
 ('Daniel', 1600, 88, 'TX'),
 ('Kelvin', 1382, 99, 'FL'),
 ('Nancy', 1442, 74, 'TX'),
 ('Pavel', 1599, 82, 'NY'),
 ('Josh', 1482, 78, 'CA'),
 ('Cynthia', 1582, 94, 'CA')]

In [8]:
sum_gpa = rdd_transformation.map(lambda x: x[2]).reduce(lambda x,y: x+y)

# view the sum
sum_gpa

843

In [9]:
sum_gpa / rdd_transformation.count()

84.3

**Associative and Commutative Properties**

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [10]:
data = [1,2,3,4,5]
for i in range(1,5):
    rdd = spark.sparkContext.parallelize(data, i)
    print('partition: ', rdd.glom().collect())
    print('addition: ', rdd.reduce(lambda a,b: a+b))

partition:  [[1, 2, 3, 4, 5]]
addition:  15
partition:  [[1, 2], [3, 4, 5]]
addition:  15
partition:  [[1], [2, 3], [4, 5]]
addition:  15
partition:  [[1], [2], [3], [4, 5]]
addition:  15


In [11]:
for i in range(1,5):
    rdd = spark.sparkContext.parallelize(data, i)
    print('partition: ', rdd.glom().collect())
    print('division: ', rdd.reduce(lambda a,b: a/b))

partition:  [[1, 2, 3, 4, 5]]
division:  0.008333333333333333
partition:  [[1, 2], [3, 4, 5]]
division:  3.3333333333333335
partition:  [[1], [2, 3], [4, 5]]
division:  1.875
partition:  [[1], [2], [3], [4, 5]]
division:  0.20833333333333331


**Accumulator & Broadcast Variables**

In [15]:
spark = SparkSession.builder.getOrCreate()
student_data = [("Chris",1523,0.72,"CA"),
                ("Jake", 1555,0.83,"NY"),
                ("Cody", 1439,0.92,"CA"),
                ("Lisa",1442,0.81,"FL"),
                ("Daniel",1600,0.88,"TX"),
                ("Kelvin",1382,0.99,"FL"),
                ("Nancy",1442,0.74,"TX"),
                ("Pavel",1599,0.82,"NY"),
                ("Josh",1482,0.78,"CA"),
                ("Cynthia",1582,0.94,"CA")]
student_rdd = spark.sparkContext.parallelize(student_data)
rdd_transformation = student_rdd.map(lambda x: (x[0], x[1], int(x[2]*100), x[3]))
states = {"NY":"New York", "CA":"California", "TX":"Texas", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)
rdd_broadcast = rdd_transformation.map(lambda x: (x[0],x[1],x[2],broadcastStates.value[x[3]]))

In [16]:
sat_1500 = spark.sparkContext.accumulator(0)

# confirm type
type(sat_1500)


pyspark.accumulators.Accumulator

In [17]:
def count_high_sat_score(r):
    if r[1] > 1500: sat_1500.add(1)

# confirm saved as a function
print(count_high_sat_score)

<function count_high_sat_score at 0x7912c2442320>


In [18]:
rdd_broadcast.foreach(lambda x: count_high_sat_score(x))

# confirm accumulator worked
print(sat_1500)

5


In [ ]:
spark.stop()
